# Kaggle New House Transactions – Baseline Notebook

This notebook implements a first-pass baseline solution for the housing transactions forecasting task. The focus is on clean data preparation, simple feature engineering, a leakage-free time-based validation split, and a single tree-based regressor to create a valid `submission.csv` file ready for Kaggle upload.


Trying to play it safe, I start by loading the main Python libraries and setting some constants. LightGBM is the model I'm planning to use, and Pandas/Numpy are obviously for the data wrangling part.


In [3]:

from __future__ import annotations

from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 150)

DATA_DIR = Path("/gpfs/helios/home/leonid98/ml-project-2025/data")
TRAIN_DIR = DATA_DIR / "train"
TARGET_COL = "amount_new_house_transactions"
SUBMISSION_TARGET_COL = "new_house_transaction_amount"


Next I list all the CSV sources and the columns I plan to keep from each table. It looks long, but it helps me stay consistent when I merge everything later.


In [4]:
MONTHLY_TABLE_KEYS = {
    "new_house": "new_house_transactions.csv",
    "new_house_nearby": "new_house_transactions_nearby_sectors.csv",
    "pre_owned": "pre_owned_house_transactions.csv",
    "pre_owned_nearby": "pre_owned_house_transactions_nearby_sectors.csv",
    "land": "land_transactions.csv",
    "land_nearby": "land_transactions_nearby_sectors.csv",
}

NEW_HOUSE_FEATURES = [
    "num_new_house_transactions",
    "area_new_house_transactions",
    "price_new_house_transactions",
    "num_new_house_available_for_sale",
    "area_new_house_available_for_sale",
    "period_new_house_sell_through",
]

NEW_HOUSE_NEARBY_FEATURES = [
    "num_new_house_transactions_nearby_sectors",
    "area_new_house_transactions_nearby_sectors",
    "price_new_house_transactions_nearby_sectors",
    "num_new_house_available_for_sale_nearby_sectors",
    "area_new_house_available_for_sale_nearby_sectors",
    "period_new_house_sell_through_nearby_sectors",
]

PRE_OWNED_FEATURES = [
    "num_pre_owned_house_transactions",
    "area_pre_owned_house_transactions",
    "price_pre_owned_house_transactions",
    "amount_pre_owned_house_transactions",
]

PRE_OWNED_NEARBY_FEATURES = [
    "num_pre_owned_house_transactions_nearby_sectors",
    "area_pre_owned_house_transactions_nearby_sectors",
    "price_pre_owned_house_transactions_nearby_sectors",
    "amount_pre_owned_house_transactions_nearby_sectors",
]

LAND_FEATURES = [
    "num_land_transactions",
    "construction_area",
    "planned_building_area",
    "transaction_amount",
]

LAND_NEARBY_FEATURES = [
    "num_land_transactions_nearby_sectors",
    "construction_area_nearby_sectors",
    "planned_building_area_nearby_sectors",
    "transaction_amount_nearby_sectors",
]

POI_FEATURES = [
    "resident_population",
    "number_of_shops",
    "catering",
    "retail",
    "transportation_station",
    "education",
    "resident_population_dense",
    "number_of_shops_dense",
]

CITY_FEATURES = [
    "gdp_100m",
    "per_capita_disposable_income_absolute_yuan",
    "real_estate_development_investment_completed_10k",
]

FEATURE_BLOCKS = [
    ("new_house", NEW_HOUSE_FEATURES),
    ("new_house_nearby", NEW_HOUSE_NEARBY_FEATURES),
    ("pre_owned", PRE_OWNED_FEATURES),
    ("pre_owned_nearby", PRE_OWNED_NEARBY_FEATURES),
    ("land", LAND_FEATURES),
    ("land_nearby", LAND_NEARBY_FEATURES),
]


def normalize_month_string(raw_value: str) -> str:
    """Convert strings like '2019-Jan' or '2019 Jan' into '2019 Jan'."""
    if pd.isna(raw_value):
        return raw_value
    value = str(raw_value).strip()
    value = value.replace("-", " ")
    value = " ".join(value.split())  # collapse duplicate spaces
    return value


def parse_month_to_datetime(raw_value: str) -> pd.Timestamp:
    normalized = normalize_month_string(raw_value)
    dt = pd.to_datetime(normalized, format="%Y %b", errors="coerce")
    if pd.isna(dt):
        dt = pd.to_datetime(normalized, errors="coerce")
    return dt


def split_id_into_components(id_series: pd.Series) -> pd.DataFrame:
    parts = id_series.str.split("_", n=1, expand=True)
    parts.columns = ["month_part", "sector"]
    parts["month"] = parts["month_part"].apply(parse_month_to_datetime)
    parts["sector"] = parts["sector"].str.strip()
    return parts[["month", "sector"]]


def mean_absolute_percentage_error(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-6) -> float:
    denom = np.maximum(np.abs(y_true), eps)
    return np.mean(np.abs(y_true - y_pred) / denom)


def build_model():
    return LGBMRegressor(random_state=42, n_estimators=1000, learning_rate=0.05, subsample=0.9, colsample_bytree=0.8)



Here I'm writing helper functions for loading and cleaning the raw CSVs. The idea is to parse the dates, tidy sector names, and keep everything in a dictionary so later steps are easier to manage.


In [5]:
def load_data(data_dir: Path = DATA_DIR) -> Dict[str, pd.DataFrame]:
    """Read all CSV files required for the baseline pipeline."""
    train_dir = data_dir / "train"
    data: Dict[str, pd.DataFrame] = {}
    for key, filename in MONTHLY_TABLE_KEYS.items():
        data[key] = pd.read_csv(train_dir / filename)
    data["sector_poi"] = pd.read_csv(train_dir / "sector_POI.csv")
    data["city_indexes"] = pd.read_csv(train_dir / "city_indexes.csv")
    data["test"] = pd.read_csv(data_dir / "test.csv")
    data["sample_submission"] = pd.read_csv(data_dir / "sample_submission.csv")
    return data


def preprocess_tables(raw_data: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """Create cleaned copies with parsed datetime columns and helper keys."""
    tables: Dict[str, pd.DataFrame] = {}

    for key in MONTHLY_TABLE_KEYS:
        df = raw_data[key].copy()
        df["month"] = df["month"].apply(parse_month_to_datetime)
        df["sector"] = df["sector"].str.strip()
        tables[key] = df

    poi = raw_data["sector_poi"].copy()
    poi["sector"] = poi["sector"].str.strip()
    tables["sector_poi"] = poi

    city_idx = raw_data["city_indexes"].copy()
    city_idx["year"] = pd.to_numeric(city_idx["city_indicator_data_year"], errors="coerce").astype("Int64")
    tables["city_indexes"] = city_idx

    tables["test"] = raw_data["test"].copy()
    tables["sample_submission"] = raw_data["sample_submission"].copy()
    return tables


This block builds the actual modeling tables: I merge all monthly tables on `(month, sector)`, add POI and city stats, engineer the simple time features, and make sure the target is zero-filled when it's missing.


All the merging utilities live here: collecting every `(month, sector)` combination, stitching features from each table, converting everything to numbers, and separating train/test with matching columns.


In [6]:
def gather_all_month_sector_pairs(tables: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    frames = []
    for key in MONTHLY_TABLE_KEYS:
        frames.append(tables[key][["month", "sector"]])
    combined = pd.concat(frames, ignore_index=True).drop_duplicates()
    combined = combined.dropna(subset=["month", "sector"]).reset_index(drop=True)
    return combined.sort_values(["month", "sector"]).reset_index(drop=True)


def build_feature_frame(base_df: pd.DataFrame, tables: Dict[str, pd.DataFrame], include_target: bool = True) -> pd.DataFrame:
    df = base_df.copy()
    df = df.dropna(subset=["month", "sector"])
    df["year"] = df["month"].dt.year.astype("Int64")
    df["month_num"] = df["month"].dt.month.astype("Int64")

    for table_name, columns in FEATURE_BLOCKS:
        available_cols = [col for col in columns if col in tables[table_name].columns]
        if not available_cols:
            continue
        merge_cols = ["month", "sector"] + available_cols
        df = df.merge(tables[table_name][merge_cols], on=["month", "sector"], how="left")

    poi_cols = [col for col in POI_FEATURES if col in tables["sector_poi"].columns]
    if poi_cols:
        df = df.merge(tables["sector_poi"][ ["sector"] + poi_cols ], on="sector", how="left")

    city_cols = [col for col in CITY_FEATURES if col in tables["city_indexes"].columns]
    if city_cols:
        city_frame = tables["city_indexes"][ ["year"] + city_cols ].drop_duplicates(subset=["year"], keep="last")
        df = df.merge(city_frame, on="year", how="left")

    if include_target:
        target_frame = tables["new_house"][ ["month", "sector", TARGET_COL] ]
        df = df.merge(target_frame, on=["month", "sector"], how="left")
        df[TARGET_COL] = df[TARGET_COL].fillna(0.0)

    return df


def numeric_feature_cleanup(df: pd.DataFrame, feature_cols: List[str]) -> pd.DataFrame:
    for col in feature_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df[feature_cols] = df[feature_cols].fillna(0.0)
    return df


def prepare_train_data(tables: Dict[str, pd.DataFrame]) -> Tuple[pd.DataFrame, List[str]]:
    base_pairs = gather_all_month_sector_pairs(tables)
    train_df = build_feature_frame(base_pairs, tables, include_target=True)
    feature_cols = [col for col in train_df.columns if col not in {"month", "sector", TARGET_COL}]
    train_df = numeric_feature_cleanup(train_df, feature_cols)
    train_df = train_df.sort_values(["month", "sector"]).reset_index(drop=True)
    return train_df, feature_cols


def prepare_test_data(tables: Dict[str, pd.DataFrame], feature_cols: List[str]) -> pd.DataFrame:
    test_df = tables["test"].copy()
    month_sector = split_id_into_components(test_df["id"])
    test_df = pd.concat([test_df, month_sector], axis=1)
    enriched = build_feature_frame(test_df[["id", "month", "sector"]], tables, include_target=False)

    for col in feature_cols:
        if col not in enriched.columns:
            enriched[col] = 0.0
    enriched = numeric_feature_cleanup(enriched, feature_cols)
    return enriched[["id", "month", "sector"] + feature_cols]



For model evaluation I go with a simple rolling-style split: keep the latest ~20% of months as validation, train on the earlier ones, and compute MAE+MAPE after clipping negatives.


In [7]:
def choose_validation_months(unique_months: np.ndarray) -> np.ndarray:
    total = len(unique_months)
    if total <= 1:
        return unique_months
    val_count = min(6, max(1, total // 5))
    if total - val_count < 1:
        val_count = max(1, total - 1)
    return unique_months[-val_count:]


def train_and_validate_model(train_df: pd.DataFrame, feature_cols: List[str]) -> Tuple[object, Dict[str, float]]:
    unique_months = train_df["month"].sort_values().unique()
    val_months = choose_validation_months(unique_months)

    train_mask = ~train_df["month"].isin(val_months)
    val_mask = train_df["month"].isin(val_months)

    if train_mask.sum() == 0 or val_mask.sum() == 0:
        raise ValueError("Validation split failed. Please ensure there are enough months of data.")

    X_train = train_df.loc[train_mask, feature_cols]
    y_train = train_df.loc[train_mask, TARGET_COL]
    X_val = train_df.loc[val_mask, feature_cols]
    y_val = train_df.loc[val_mask, TARGET_COL]

    model = build_model()
    model.fit(X_train, y_train)

    val_preds = np.clip(model.predict(X_val), 0, None)
    metrics = {
        "mae": mean_absolute_error(y_val, val_preds),
        "mape": mean_absolute_percentage_error(y_val.values, val_preds),
        "val_months": val_months,
    }
    return model, metrics


def train_on_full_data_and_predict(train_df: pd.DataFrame, test_df: pd.DataFrame, feature_cols: List[str]) -> Tuple[object, pd.DataFrame]:
    model = build_model()
    model.fit(train_df[feature_cols], train_df[TARGET_COL])
    test_preds = np.clip(model.predict(test_df[feature_cols]), 0, None)

    submission = pd.DataFrame({
        "id": test_df["id"],
        SUBMISSION_TARGET_COL: test_preds,
    })
    submission.to_csv("submission.csv", index=False)
    return model, submission


Finally I actually run the whole pipeline: load the data, build features, check the validation months, and then fit on all the data to spit out `submission.csv` so I can upload it to Kaggle.


In [8]:
# Load every CSV once and normalize shared tables.
raw_data = load_data()
tables = preprocess_tables(raw_data)

# Build the modeling tables with aligned features for train/test.
train_df, feature_cols = prepare_train_data(tables)
test_df = prepare_test_data(tables, feature_cols)

print(f"Training rows: {len(train_df):,}, features: {len(feature_cols)}")
print(f"Test rows: {len(test_df):,}")

# Time-aware validation to sanity-check the baseline model.
_, val_metrics = train_and_validate_model(train_df, feature_cols)
val_month_labels = [pd.to_datetime(m).strftime("%Y-%b") for m in val_metrics["val_months"]]
print("Validation months:", val_month_labels)
print(f"Validation MAE: {val_metrics['mae']:.2f}")
print(f"Validation MAPE: {val_metrics['mape']:.4f}")

# Retrain on the full dataset and export a submission file.
full_model, submission = train_on_full_data_and_predict(train_df, test_df, feature_cols)
print("Saved submission.csv with shape:", submission.shape)
submission.head()


Training rows: 6,432, features: 41
Test rows: 1,152
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000737 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6880
[LightGBM] [Info] Number of data points in the train set: 5856, number of used features: 41
[LightGBM] [Info] Start training from score 27919.951946
Validation months: ['2024-Feb', '2024-Mar', '2024-Apr', '2024-May', '2024-Jun', '2024-Jul']
Validation MAE: 1250.68
Validation MAPE: 3201167.1658
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6883
[LightGBM] [Info] Number of data points in the train set: 6432, number of used features: 41
[LightGBM] [Info] Start training from score 27526.152219
Saved submission.csv with shape: (1152, 2)


,id,new_house_transaction_amount
0,2024 Aug_sector 1,7.516924
1,2024 Aug_sector 2,0.000000
2,2024 Aug_sector 3,23.089488
3,2024 Aug_sector 4,24.495626
4,2024 Aug_sector 5,0.000000


In [ ]:
!kaggle competitions submit -c china-real-estate-demand-prediction -f submission.csv -m "Baseline submission"

403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/competitions/submission-url
